# Problem 3

Use this notebook to write your code for problem 3.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3D - Convolutional network

As in problem 2, we have conveniently provided for your use code that loads, preprocesses, and deals with the uglies of the MNIST data.

In [2]:
# load MNIST data into Keras format
import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
# look at the shapes
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
# we'll need to one-hot encode the labels
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)

In [5]:
# don't forget to NORMALIZE
x_train = np.divide(x_train, 255)
x_test = np.divide(x_test, 255)

In [6]:
# we must reshape the X data (add a channel dimension)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
x_test = x_test.reshape(tuple(list(x_test.shape) + [1]))

In [7]:
# look at the shapes
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers

# sample model
# note: what is the difference between 'same' and 'valid' padding?
# Take a look at the outputs to understand the difference, or read the Keras documentation!
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))

model.add(Conv2D(8, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [9]:
# why don't we take a look at the layers and outputs
# note: `None` in the first dimension means it can take any batch_size!
for i in range(len(model.layers)):
    layer = model.layers[i]
    print(layer)
    print(layer.output_shape)

(None, 28, 28, 8)
(None, 28, 28, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 7, 7, 8)
(None, 7, 7, 8)
(None, 392)
(None, 64)
(None, 64)
(None, 10)
(None, 10)


In [10]:
# our model has some # of parameters:
model.count_params()

26466

In [11]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model, iterating on the data in batches of 32 samples
history = model.fit(x_train, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 30s 508us/step - loss: 0.6052 - acc: 0.8053 - val_loss: 0.1796 - val_acc: 0.9491

Above, we output the training loss/accuracy as well as the validation (here, the TEST) loss and accuracy. To confirm that these are right, we can explicitly print out the training and test losses/accuracies.

In [13]:
# note that our model outputs two eval params:
# 1. loss (categorical cross-entropy)
# 2. accuracy
model.metrics_names

['loss', 'acc']

In [14]:
model.evaluate(x=x_train, y=y_train)

60000/60000 [==============================] - 11s 186us/step

[0.07214776885223885, 0.9790333333333333]

In [15]:
model.evaluate(x=x_test, y=y_test)

10000/10000 [==============================] - 2s 224us/step

[0.06971469139587134, 0.9795]

Not bad! Let's see if you can do better.

In [28]:
model2 = Sequential()
model2.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(28, 28, 1)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.1))

model2.add(Conv2D(8, (3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())

model2.add(Flatten())
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dense(10))
model2.add(Activation('softmax'))

# For a multi-class classification problem
model2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history2 = model2.fit(x_train, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 31s 522us/step - loss: 0.2280 - acc: 0.9281 - val_loss: 0.0784 - val_acc: 0.9744

In [29]:
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.03665039017227246
Test accuracy: 0.9873


In [24]:
model2.count_params()

26466

In [15]:
model3 = Sequential()
model3.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(28, 28, 1)))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(1))

model3.add(Conv2D(8, (3, 3), padding='same'))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(1))
model3.add(BatchNormalization())

model3.add(Flatten())
model3.add(Dense(64))
model3.add(Activation('relu'))
model3.add(Dropout(1))


model3.add(Dense(10))
model3.add(Activation('softmax'))

# For a multi-class classification problem
model3.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history3 = model3.fit(x_train, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 29s 490us/step - loss: 0.1503 - acc: 0.9537 - val_loss: 0.0719 - val_acc: 0.9776

KeyboardInterrupt: 

In [14]:
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0723223465348247
Test accuracy: 0.985
